In [ ]:
#Modelo para predecir tendencia mensual (clasificador ML: Sube o baja)
#Necesitamos muchos mas datos, de varias acciones: entrenamos el modelo pej en los años 2020-2023 y testeamos en el 24
import pickle
import pandas as pd

with open("ibex_cache.pkl", "rb") as f:
    data = pickle.load(f)

price_data = data["price_data"]
info_meta = data["info_meta"]

# Coger un ticker cualquiera
sample_ticker = list(price_data.keys())[0]
sample = price_data[sample_ticker]

print("Ticker de ejemplo:", sample_ticker)
print("Tipo interno:", type(sample))
print("Contenido ejemplo:")
print(sample)


Ticker de ejemplo: ACS.MC
Tipo interno: <class 'pandas.core.frame.DataFrame'>
Contenido ejemplo:
                                Open       High        Low      Close  \
Date                                                                    
2024-12-02 00:00:00+01:00  43.840000  44.380001  43.840000  44.200001   
2024-12-03 00:00:00+01:00  44.500000  46.599998  44.480000  46.599998   
2024-12-04 00:00:00+01:00  46.459999  47.099998  46.360001  47.020000   
2024-12-05 00:00:00+01:00  47.099998  48.099998  47.099998  48.020000   
2024-12-06 00:00:00+01:00  47.880001  48.200001  47.880001  48.000000   
...                              ...        ...        ...        ...   
2025-11-25 00:00:00+01:00  77.099998  79.000000  76.500000  78.300003   
2025-11-26 00:00:00+01:00  79.000000  80.349998  78.599998  79.900002   
2025-11-27 00:00:00+01:00  80.099998  80.500000  79.500000  79.500000   
2025-11-28 00:00:00+01:00  79.449997  80.150002  78.949997  79.650002   
2025-12-01 00:00:00+01:00  

In [26]:
import pandas as pd

panel = pd.concat(price_data, axis=1)  # columnas MultiIndex (ticker, campo)
panel.index = pd.to_datetime(panel.index)
panel = panel.sort_index()

print(panel.columns.levels)
print(panel.head())


[['ACS.MC', 'ACX.MC', 'AENA.MC', 'AMS.MC', 'ANA.MC', 'ANE.MC', 'BBVA.MC', 'BKT.MC', 'CABK.MC', 'CLNX.MC', 'COL.MC', 'ELE.MC', 'ENG.MC', 'FDR.MC', 'FER.MC', 'GRF.MC', 'IAG.MC', 'IBE.MC', 'ITX.MC', 'LOG.MC', 'MAP.MC', 'MRL.MC', 'MTS.MC', 'NTGY.MC', 'PUIG.MC', 'RED.MC', 'SAB.MC', 'SAN.MC', 'TEF.MC', 'UNI.MC'], ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits']]
                              ACS.MC                                   \
                                Open       High        Low      Close   
Date                                                                    
2024-12-02 00:00:00+01:00  43.840000  44.380001  43.840000  44.200001   
2024-12-03 00:00:00+01:00  44.500000  46.599998  44.480000  46.599998   
2024-12-04 00:00:00+01:00  46.459999  47.099998  46.360001  47.020000   
2024-12-05 00:00:00+01:00  47.099998  48.099998  47.099998  48.020000   
2024-12-06 00:00:00+01:00  47.880001  48.200001  47.880001  48.000000   

                    

In [27]:
ticker = "ACS.MC"
df = price_data[ticker].copy()

# Asegurar formato
df.index = pd.to_datetime(df.index)
df = df.sort_index()

print(df.head())


                                Open       High        Low      Close  \
Date                                                                    
2024-12-02 00:00:00+01:00  43.840000  44.380001  43.840000  44.200001   
2024-12-03 00:00:00+01:00  44.500000  46.599998  44.480000  46.599998   
2024-12-04 00:00:00+01:00  46.459999  47.099998  46.360001  47.020000   
2024-12-05 00:00:00+01:00  47.099998  48.099998  47.099998  48.020000   
2024-12-06 00:00:00+01:00  47.880001  48.200001  47.880001  48.000000   

                           Adj Close  Volume  Dividends  Stock Splits  
Date                                                                   
2024-12-02 00:00:00+01:00  42.632236  410302        0.0           0.0  
2024-12-03 00:00:00+01:00  44.947109  612698        0.0           0.0  
2024-12-04 00:00:00+01:00  45.352215  402104        0.0           0.0  
2024-12-05 00:00:00+01:00  46.316742  296316        0.0           0.0  
2024-12-06 00:00:00+01:00  46.297455  286970        0.0 

In [28]:
import numpy as np
import pandas as pd

price_col = "Adj Close"

quant = pd.DataFrame(index=df.index)
quant["price"] = df[price_col]

quant["ret_simple"] = quant["price"].pct_change()
quant["ret_log"] = np.log(quant["price"] / quant["price"].shift(1))

quant = quant.dropna()

print(quant.head())
print(quant["ret_simple"].describe())


                               price  ret_simple   ret_log
Date                                                      
2024-12-03 00:00:00+01:00  44.947109    0.054299  0.052876
2024-12-04 00:00:00+01:00  45.352215    0.009013  0.008973
2024-12-05 00:00:00+01:00  46.316742    0.021267  0.021044
2024-12-06 00:00:00+01:00  46.297455   -0.000416 -0.000417
2024-12-09 00:00:00+01:00  45.757313   -0.011667 -0.011735
count    254.000000
mean       0.002530
std        0.016806
min       -0.076628
25%       -0.005856
50%        0.003011
75%        0.011426
max        0.071081
Name: ret_simple, dtype: float64


In [29]:
quant["sma20"] = quant["price"].rolling(20).mean()
quant["sma50"] = quant["price"].rolling(50).mean()


In [30]:
quant["signal"] = (quant["sma20"] > quant["sma50"]).astype(int)


In [31]:
quant["strategy_ret"] = quant["signal"].shift(1) * quant["ret_simple"]


In [32]:
quant["equity_curve"] = (1 + quant["strategy_ret"]).cumprod()
quant["buy_hold"] = (1 + quant["ret_simple"]).cumprod()


In [33]:
sharpe = quant["strategy_ret"].mean() / quant["strategy_ret"].std() * (252**0.5)
print("Sharpe:", sharpe)


Sharpe: 1.554413681756869


In [34]:
roll_max = quant["equity_curve"].cummax()
quant["drawdown"] = quant["equity_curve"] / roll_max - 1
print("Max DD:", quant["drawdown"].min())


Max DD: -0.16699551309812466


In [35]:
quant["signal"].value_counts()

signal
1    177
0     77
Name: count, dtype: int64

In [36]:
quant["equity_curve"].tail(1)

Date
2025-12-01 00:00:00+01:00    1.409387
Name: equity_curve, dtype: float64

In [37]:
import numpy as np

buy_hold_ret = quant["ret_simple"]
quant["buy_hold_equity"] = (1 + buy_hold_ret).cumprod()

print("Buy & hold final:", quant["buy_hold_equity"].iloc[-1])

sh_bh = buy_hold_ret.mean() / buy_hold_ret.std() * np.sqrt(252)
print("Sharpe buy&hold:", sh_bh)

roll_max_bh = quant["buy_hold_equity"].cummax()
dd_bh = quant["buy_hold_equity"] / roll_max_bh - 1
print("Max DD buy&hold:", dd_bh.min())


Buy & hold final: 1.8331199105924079
Sharpe buy&hold: 2.389419731868193
Max DD buy&hold: -0.16699551309812422


In [38]:
cost = 0.001  # 0.1%

quant["trade"] = quant["signal"].diff().abs()  # 1 cuando hay cambio de posición
quant["strategy_ret_tc"] = quant["strategy_ret"] - quant["trade"] * cost
quant["equity_curve_tc"] = (1 + quant["strategy_ret_tc"]).cumprod()

print("Equity con TC:", quant["equity_curve_tc"].iloc[-1])


Equity con TC: 1.4023446705773714


In [39]:
import numpy as np
import pandas as pd

results = []

for short in [5,10,15,20,30]:
    for long in [20,40,60,80,100]:
        if short >= long:
            continue
        
        tmp = quant.copy()
        tmp["sma_s"] = tmp["price"].rolling(short).mean()
        tmp["sma_l"] = tmp["price"].rolling(long).mean()
        tmp["signal"] = (tmp["sma_s"] > tmp["sma_l"]).astype(int)

        tmp["strategy_ret"] = tmp["signal"].shift(1) * tmp["ret_simple"]
        tmp = tmp.dropna()

        sharpe = tmp["strategy_ret"].mean() / tmp["strategy_ret"].std() * np.sqrt(252)
        final = (1 + tmp["strategy_ret"]).cumprod().iloc[-1]

        results.append((short, long, sharpe, final))

df_res = pd.DataFrame(results, columns=["short", "long", "sharpe", "final"])
print(df_res.sort_values("sharpe", ascending=False).head(10))


    short  long    sharpe     final
22     30   100  2.759746  1.476911
9      10   100  2.759746  1.476911
18     20   100  2.759746  1.476911
14     15   100  2.759746  1.476911
4       5   100  2.759746  1.476911
0       5    20  2.192282  1.471529
21     30    80  2.082580  1.445007
17     20    80  2.082580  1.445007
16     20    60  1.910714  1.455758
3       5    80  1.778149  1.348487


In [40]:
import pandas as pd

panel = pd.concat(price_data, axis=1)  # columnas MultiIndex: (ticker, campo)
panel.index = pd.to_datetime(panel.index)
panel = panel.sort_index()

# Nos quedamos solo con Adj Close para cada ticker
adj_close = panel.xs('Adj Close', level=1, axis=1)

print(adj_close.head())
print(adj_close.columns)


                              ACS.MC    ACX.MC     AENA.MC     AMS.MC  \
Date                                                                    
2024-12-02 00:00:00+01:00  42.632236  8.947750  207.267868  66.400375   
2024-12-03 00:00:00+01:00  44.947109  8.980715  208.064285  67.557037   
2024-12-04 00:00:00+01:00  45.352215  9.117286  209.458023  67.459015   
2024-12-05 00:00:00+01:00  46.316742  9.343334  204.281296  68.654892   
2024-12-06 00:00:00+01:00  46.297455  9.399845  203.086670  68.615685   

                               ANA.MC     ANE.MC   BBVA.MC    BKT.MC  \
Date                                                                   
2024-12-02 00:00:00+01:00  115.823692  19.844772  8.478012  7.131157   
2024-12-03 00:00:00+01:00  114.470154  19.355986  8.642358  7.279882   
2024-12-04 00:00:00+01:00  113.503349  19.277781  8.729254  7.241747   
2024-12-05 00:00:00+01:00  113.503349  19.072489  9.097617  7.432420   
2024-12-06 00:00:00+01:00  114.373474  19.551502  9.0239

In [41]:
# Precio de cierre de cada mes (último día hábil del mes)
monthly_prices = adj_close.resample("M").last()

# Retornos mensuales simples
monthly_ret = monthly_prices.pct_change()

print(monthly_prices.tail())
print(monthly_ret.tail())


                              ACS.MC  ACX.MC    AENA.MC     AMS.MC  \
Date                                                                 
2025-08-31 00:00:00+02:00  64.550003   10.55  24.740000  71.599998   
2025-09-30 00:00:00+02:00  68.000000   11.10  23.270000  67.500000   
2025-10-31 00:00:00+01:00  71.199997   11.18  23.549999  66.320000   
2025-11-30 00:00:00+01:00  79.650002   12.17  23.459999  63.340000   
2025-12-31 00:00:00+01:00  78.150002   12.13  23.350000  63.060001   

                               ANA.MC     ANE.MC    BBVA.MC     BKT.MC  \
Date                                                                     
2025-08-31 00:00:00+02:00  169.199997  22.980000  15.207399  12.468122   
2025-09-30 00:00:00+02:00  170.899994  22.040001  16.041893  13.113783   
2025-10-31 00:00:00+01:00  192.000000  23.860001  17.116915  12.786061   
2025-11-30 00:00:00+01:00  172.899994  21.080000  18.565001  13.560000   
2025-12-31 00:00:00+01:00  171.199997  20.820000  18.645000  13.5

C:\Users\USUARI\AppData\Local\Temp\ipykernel_7720\3731744632.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_prices = adj_close.resample("M").last()


In [42]:
momentum_6m = monthly_prices.pct_change(6)

print(momentum_6m.tail())


                             ACS.MC    ACX.MC   AENA.MC    AMS.MC    ANA.MC  \
Date                                                                          
2025-08-31 00:00:00+02:00  0.277479 -0.040496 -0.884305 -0.003009  0.474379   
2025-09-30 00:00:00+02:00  0.324071  0.056219 -0.892084 -0.030502  0.463304   
2025-10-31 00:00:00+01:00  0.327252  0.157526 -0.893631 -0.029275  0.546663   
2025-11-30 00:00:00+01:00  0.414189  0.202532 -0.901013 -0.125958  0.259407   
2025-12-31 00:00:00+01:00  0.361664  0.153162  0.030450 -0.106691  0.158884   

                             ANE.MC   BBVA.MC    BKT.MC   CABK.MC   CLNX.MC  \
Date                                                                          
2025-08-31 00:00:00+02:00  0.298020  0.255412  0.401118  0.332178 -0.117149   
2025-09-30 00:00:00+02:00  0.330123  0.353860  0.342634  0.302498 -0.101281   
2025-10-31 00:00:00+01:00  0.484631  0.445688  0.293935  0.360962 -0.241198   
2025-11-30 00:00:00+01:00  0.119603  0.431491  0.23

In [43]:
# Rank descendente: 1 = mejor momentum
ranks = momentum_6m.rank(axis=1, ascending=False)

print(ranks.tail())


                           ACS.MC  ACX.MC  AENA.MC  AMS.MC  ANA.MC  ANE.MC  \
Date                                                                         
2025-08-31 00:00:00+02:00     9.0    26.0     30.0    24.0     2.0     8.0   
2025-09-30 00:00:00+02:00    11.0    21.0     30.0    26.0     3.0    10.0   
2025-10-31 00:00:00+01:00    10.0    18.0     30.0    26.0     2.0     3.0   
2025-11-30 00:00:00+01:00     2.0    11.0     30.0    26.0     8.0    19.0   
2025-12-31 00:00:00+01:00     3.0    13.0     21.0    25.0    11.0    20.0   

                           BBVA.MC  BKT.MC  CABK.MC  CLNX.MC  ...  MAP.MC  \
Date                                                          ...           
2025-08-31 00:00:00+02:00     10.0     4.0      6.0     28.0  ...     3.0   
2025-09-30 00:00:00+02:00      7.0     8.0     13.0     28.0  ...     4.0   
2025-10-31 00:00:00+01:00      5.0    12.0      9.0     29.0  ...    15.0   
2025-11-30 00:00:00+01:00      1.0     9.0      6.0     29.0  ...   

In [44]:
top_n = 5
longs = ranks <= top_n   # DataFrame booleano


In [45]:
# Shift para evitar look-ahead bias: usamos la señal de mes anterior
signal_long = longs.shift(1)

# Retorno de la estrategia = media de retornos de los activos en cartera
strategy_ret = (signal_long * monthly_ret).sum(axis=1) / signal_long.sum(axis=1)

# Si algún mes no hay activos seleccionados (por NaNs), mejor poner 0
strategy_ret = strategy_ret.fillna(0.0)

equity_cs = (1 + strategy_ret).cumprod()

print(strategy_ret.describe())
print("Equity final cross-sectional:", equity_cs.iloc[-1])


ZeroDivisionError: division by zero

In [46]:
# Cartera equiponderada: todos los activos con igual peso
ew_ret = monthly_ret.mean(axis=1)
ew_equity = (1 + ew_ret).cumprod()

print("Equity final EW:", ew_equity.iloc[-1])


Equity final EW: 1.3268793561746106
